In [693]:
# Import required libraries and dependencies
import pandas as pd
import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
# warnings.simplefilter(action='ignore', category=DeprecationWarning)
# warnings.simplefilter(action='ignore', category=RuntimeWarning)

In [694]:
# Load the data into a Pandas DataFrame
df_market_data = pd.read_csv("./Resources/crypto_market_data.csv",index_col="coin_id")

# Display sample data
df_market_data.head()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,1.08388,7.60278,6.57509,7.67258,-3.25185,83.51840,37.51761
ethereum,0.22392,10.38134,4.80849,0.13169,-12.88890,186.77418,101.96023
tether,-0.21173,0.04935,0.00640,-0.04237,0.28037,-0.00542,0.01954
ripple,-0.37819,-0.60926,2.24984,0.23455,-17.55245,39.53888,-16.60193
bitcoin-cash,2.90585,17.09717,14.75334,15.74903,-13.71793,21.66042,14.49384


In [695]:
# Generate summary statistics
df_market_data.describe()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
count,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000
mean,-0.269686,4.497147,0.185787,1.545693,-0.094119,236.537432,347.667956
std,2.694793,6.375218,8.376939,26.344218,47.365803,435.225304,1247.842884
min,-13.527860,-6.094560,-18.158900,-34.705480,-44.822480,-0.392100,-17.567530
25%,-0.608970,0.047260,-5.026620,-10.438470,-25.907990,21.660420,0.406170
50%,-0.063410,3.296410,0.109740,-0.042370,-7.544550,83.905200,69.691950
75%,0.612090,7.602780,5.510740,4.578130,0.657260,216.177610,168.372510
max,4.840330,20.694590,24.239190,140.795700,223.064370,2227.927820,7852.089700


In [696]:
# Plot your data to see what's in your DataFrame
df_market_data.hvplot.line(
    width=800,
    height=400,
    rot=90
)


:NdOverlay   [Variable]
   :Curve   [coin_id]   (value)

### Prepare the Data

In [697]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
rate_df_scaled = StandardScaler().fit_transform(df_market_data[["price_change_percentage_24h" , "price_change_percentage_7d" , "price_change_percentage_14d" , "price_change_percentage_30d", "price_change_percentage_60d" \
                                                                 , "price_change_percentage_200d", "price_change_percentage_1y"]])
rate_df_scaled[0:5]

array([[ 0.50852937,  0.49319307,  0.77220043,  0.23545963, -0.0674951 ,
        -0.35595348, -0.25163688],
       [ 0.18544589,  0.93444504,  0.55869212, -0.05434093, -0.27348273,
        -0.11575947, -0.19935211],
       [ 0.02177396, -0.70633685, -0.02168042, -0.06103015,  0.00800452,
        -0.55024692, -0.28206051],
       [-0.04076438, -0.81092807,  0.24945797, -0.05038797, -0.37316402,
        -0.45825882, -0.29554614],
       [ 1.19303608,  2.00095907,  1.76061001,  0.54584206, -0.29120287,
        -0.49984776, -0.27031695]])

In [698]:
# Create a DataFrame with the scaled data
scaled_data = pd.DataFrame(rate_df_scaled, columns=["price_change_percentage_24h", "price_change_percentage_7d", "price_change_percentage_14d", "price_change_percentage_30d", "price_change_percentage_60d", "price_change_percentage_200d", "price_change_percentage_1y"])

# # Copy the crypto names from the original data #Omited this step as it caused issues in the calculation. 
# scaled_data["Ticker"] = df_market_data.index
# # Set the coinid column as index
# scaled_data.set_index('Ticker')

# Display sample data
scaled_data.head()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
0,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637
1,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352
2,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061
3,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546
4,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317


---

### Find the Best Value for k Using the Original Data.

In [699]:
# Create a list with the number of k-values from 1 to 11
k = list(range(1, 12))

In [700]:
# Create an empty list to store the inertia values
inertia = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_scaled`
# 3. Append the model.inertia_ to the inertia list
for i in k:
    model = KMeans(n_clusters=i, random_state=0)
    model.fit(rate_df_scaled)
    inertia.append(model.inertia_)


In [701]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data_pca = {
    "k": k,
    "inertia": inertia
}

# Create a DataFrame with the data to plot the Elbow curve
df_elbow_pca = pd.DataFrame(elbow_data_pca)

In [702]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
elbow_plot_pca = df_elbow_pca.hvplot.line(x="k", y="inertia", title="Elbow Curve Using PCA Data", xticks=k)
elbow_plot_pca

:Curve   [k]   (inertia)

#### Answer the following question: 

**Question:** What is the best value for `k`?

**Answer: The best value for K is 4** 

---

### Cluster Cryptocurrencies with K-means Using the Original Data

In [703]:
# Initialize the K-Means model using the best value for k
model = KMeans(n_clusters=4)

In [704]:
# Fit the K-Means model using the scaled data
model.fit(scaled_data)

KMeans(n_clusters=4)

In [705]:
# Predict the clusters to group the cryptocurrencies using the scaled data
clusters = model.predict(scaled_data)
# Print the resulting array of cluster values.
print(clusters)

[0 0 3 3 0 0 0 0 0 3 3 3 3 0 3 0 3 3 0 3 3 0 3 3 3 3 3 3 0 3 3 3 1 0 3 3 2
 3 3 3 3]


In [706]:
# Create a copy of the DataFrame
clusters_predict = clusters.copy()

In [707]:
# Add a new column to the DataFrame with the predicted clusters
scaled_data["predicted_clusters"] = clusters

# Display sample data
scaled_data.head()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y,predicted_clusters
0,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637,0
1,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352,0
2,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061,3
3,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546,3
4,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317,0


In [708]:
# Create a scatter plot using hvPlot by setting 
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`. 
# scaled_data["Ticker"] = df_market_data.index
# # Set the coinid column as index
# scaled_data.set_index('Ticker')

# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point. scaled_data  df_market_data
scaled_data.hvplot.scatter(
    width=800,
    height=400,
    rot=90,
    x="price_change_percentage_24h",
    y="price_change_percentage_7d",
    hover_cols="coin_id",
    by="predicted_clusters",
)


:NdOverlay   [predicted_clusters]
   :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d)

---

### Optimize Clusters with Principal Component Analysis.

In [709]:
# Create a PCA model instance and set `n_components=3`.
pca = PCA(n_components=3)

In [710]:
# Use the PCA model with `fit_transform` to reduce to 
# three principal components.
clusters_pca = pca.fit_transform(scaled_data)
# View the first five rows of the DataFrame. 
clusters_pca[0:5]

array([[ 2.03409288, -0.64157137, -0.35762863],
       [ 1.99450168, -0.49054937, -0.91224962],
       [-1.06854412, -0.41720452,  0.5963108 ],
       [-1.04091865, -0.45523427,  0.4678008 ],
       [ 3.30276384, -1.24245766, -0.217276  ]])

In [711]:
# Retrieve the explained variance to determine how much information
# can be attributed to each principal component.
'''
https://stackoverflow.com/questions/32857029/python-scikit-learn-pca-explained-variance-ratio-cutoff 
pca.explained_variance_ratio_ parameter returns a vector of the variance explained by each dimension. Thus pca.explained_variance_ratio_[i] gives the variance explained solely by the i+1st dimension
pca.explained_variance_ratio_.cumsum(). That will return a vector x such that x[i] returns the cumulative variance explained by the first i+1 dimensions.
'''
# pca.explained_variance_ratio_.cumsum() # returns array([0.36941241, 0.66110616, 0.89004494])
pca.explained_variance_ratio_ # returns array([0.36941241, 0.29169375, 0.22893878])
#  

array([0.36941241, 0.29169375, 0.22893878])

#### Answer the following question: 

**Question:** What is the total explained variance of the three principal components?

**Answer:** 

In [712]:
# Create a new DataFrame with the PCA data.
cluster_pca_df = pd.DataFrame(clusters_pca,columns = ["PCA1", "PCA2", "PCA3"])
# Creating a DataFrame with the PCA data
cluster_pca_df
# Copy the crypto names from the original data
cluster_pca_df['coin_id'] = list(df_market_data.index)
cluster_pca_df
# Set the coinid column as index
cluster_pca_df = cluster_pca_df.set_index('coin_id')
# Display sample data
cluster_pca_df

,PCA1,PCA2,PCA3
coin_id,,,
bitcoin,2.034093,-0.641571,-0.357629
ethereum,1.994502,-0.490549,-0.912250
tether,-1.068544,-0.417205,0.596311
ripple,-1.040919,-0.455234,0.467801
bitcoin-cash,3.302764,-1.242458,-0.217276
binancecoin,2.505692,-0.575832,-0.129389
chainlink,2.770042,-0.495691,-1.890371
cardano,2.372449,-0.389146,-0.992513
litecoin,1.832344,-0.678797,-0.741581


---

### Find the Best Value for k Using the PCA Data

In [713]:
# Create a list with the number of k-values from 1 to 11
k = list(range(1,12))

In [714]:
# Create an empty list to store the inertia values
inertia_pca = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_pca`
# 3. Append the model.inertia_ to the inertia list
for i in k:
    k_model = KMeans(n_clusters=i, random_state=1)
    k_model.fit(cluster_pca_df)
    inertia_pca.append(k_model.inertia_)
inertia_pca

[325.7347395785624,
 201.28762245654414,
 111.73680938677687,
 40.197284424059305,
 29.514810382341864,
 20.988916219540823,
 15.365024356826586,
 13.055816514033344,
 9.573032652919258,
 7.938234563924161,
 6.596794530981119]

In [715]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data_pca = {"k": k, "inertia": inertia_pca}
# Create a DataFrame with the data to plot the Elbow curve
elbow_data_pca_df = pd.DataFrame(elbow_data_pca)
elbow_data_pca_df

,k,inertia
0,1,325.734740
1,2,201.287622
2,3,111.736809
3,4,40.197284
4,5,29.514810
5,6,20.988916
6,7,15.365024
7,8,13.055817
8,9,9.573033
9,10,7.938235


In [716]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
elbow_data_pca_df.hvplot.line(
    x="k", 
    y="inertia", 
    title="Elbow Curve", 
    xticks=k
)

:Curve   [k]   (inertia)

#### Answer the following questions: 

* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer: The best value for 'k' is 4 **


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer: It did not change, the best 'k' value was also 4 in the original data** 

### Cluster Cryptocurrencies with K-means Using the PCA Data

In [717]:
# Initialize the K-Means model using the best value for k
model = KMeans(n_clusters=4, random_state=1)

In [718]:
# Fit the K-Means model using the PCA data
model.fit(cluster_pca_df)

KMeans(n_clusters=4, random_state=1)

In [719]:
# Predict the clusters to group the cryptocurrencies using the PCA data
clusters_predict = model.predict(cluster_pca_df)
# Print the resulting array of cluster values.

print(clusters_predict)

[1 1 0 0 1 1 1 1 1 0 0 0 0 1 0 1 0 0 1 0 0 1 0 0 0 0 0 0 1 0 0 0 2 1 0 0 3
 0 0 0 0]


In [720]:
# Create a copy of the DataFrame with the PCA data

cluster_pca_df_copy= cluster_pca_df.copy()
# Add a new column to the DataFrame with the predicted clusters
cluster_pca_df_copy['predicted_clusters'] = clusters_predict

# Display sample data
cluster_pca_df_copy

,PCA1,PCA2,PCA3,predicted_clusters
coin_id,,,,
bitcoin,2.034093,-0.641571,-0.357629,1
ethereum,1.994502,-0.490549,-0.912250,1
tether,-1.068544,-0.417205,0.596311,0
ripple,-1.040919,-0.455234,0.467801,0
bitcoin-cash,3.302764,-1.242458,-0.217276,1
binancecoin,2.505692,-0.575832,-0.129389,1
chainlink,2.770042,-0.495691,-1.890371,1
cardano,2.372449,-0.389146,-0.992513,1
litecoin,1.832344,-0.678797,-0.741581,1


In [721]:
# Create a scatter plot using hvPlot by setting 
# `x="PC1"` and `y="PC2"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
cluster_pca_df_copy.hvplot.scatter(
    x="PCA1",
    y="PCA2",
    by = 'predicted_clusters',
    hover_cols='coin_id'
)

:NdOverlay   [predicted_clusters]
   :Scatter   [PCA1]   (PCA2,coin_id)

### Visualize and Compare the Results

In this section, you will visually analyze the cluster analysis results by contrasting the outcome with and without using the optimization techniques.

In [722]:
# Composite plot to contrast the Elbow curves
df_elbow_pca.hvplot.line(
    x="k", 
    y="inertia", 
    title= "Elbow Curve ", 
    xticks=k
    ) + elbow_data_pca_df.hvplot.line(
        x="k", y="inertia", 
        title="Elbow Curve 1", 
        xticks=k
        )


:Layout
   .Curve.I  :Curve   [k]   (inertia)
   .Curve.II :Curve   [k]   (inertia)

In [724]:
# Composite plot to contrast the clusters
# YOUR CODE HERE!
scaled_data.hvplot.scatter(
    width=800,
    height=400,
    rot=90,
    x="price_change_percentage_24h",
    y="price_change_percentage_7d",
    hover_cols="coin_id",
    by="predicted_clusters",
) + cluster_pca_df_copy.hvplot.scatter(
    width=800,
    height=400,
    rot=90,
    x="PCA1",
    y="PCA2",
    by = 'predicted_clusters',
    hover_cols='coin_id'
)


:Layout
   .NdOverlay.I  :NdOverlay   [predicted_clusters]
      :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d)
   .NdOverlay.II :NdOverlay   [predicted_clusters]
      :Scatter   [PCA1]   (PCA2,coin_id)

#### Answer the following question: 

  * **Question:** After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer: Using the dimensionally reducted PCA method we got more defined group of clusters.** 